# SQL query from table names

In This notebook we are going to test if using just the name of the table, and a shord definition of its contect we can use a model like GTP3.5-Turbo to select which tables are necessary to create a SQL Order to answer the user petition.

In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [2]:
#Functio to call the model.
def return_OAI(user_message):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)
    context = []
    context.append({'role':'system', "content": user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=context,
            temperature=0,
        )

    return (response.choices[0].message.content)

In [4]:
#Definition of the tables.
import pandas as pd

# Table and definitions sample
data = {'table': ['employees', 'salary', 'studies'],
        'definition': ['Employee information, name...',
                       'Salary details for each year',
                       'Educational studies, name of the institution, type of studies, level']}
df = pd.DataFrame(data)
print(df)

       table                                         definition
0  employees                      Employee information, name...
1     salary                       Salary details for each year
2    studies  Educational studies, name of the institution, ...


In [5]:
text_tables = '\n'.join([f"{row['table']}: {row['definition']}" for index, row in df.iterrows()])

In [6]:
print(text_tables)

employees: Employee information, name...
salary: Salary details for each year
studies: Educational studies, name of the institution, type of studies, level


In [7]:
prompt_question_tables = """
Given the following tables and their content definitions,
###Tables
{tables}

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a json format.
###User Questyion:
{question}
"""


In [9]:
#Creating the prompt, with the user questions and the tables definitions.
pqt1 = prompt_question_tables.format(tables=text_tables, question="What is the average salary of all employees?")


In [10]:
print(return_OAI(pqt1))

```json
{
    "tables": ["salary"]
}
```


In [14]:
pqt2 = prompt_question_tables.format(
    tables=text_tables,
    question="Which employees with a university degree earn more than 50000 a year?"
)


In [15]:
print(return_OAI(pqt2))

{
  "tables": ["employees", "salary", "studies"]
}


In [12]:
pqt3 = prompt_question_tables.format(tables=text_tables,
                                     question="Give me a full profile of each employee including their salary history and education background."
                                     )


In [13]:
print(return_OAI(pqt3))

```json
{
    "tables": ["employees", "salary", "studies"]
}
```


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try a few versions if you have time
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

### Overview
In this lab we tested whether GPT-3.5 can correctly identify which tables are needed to answer a question, using only table names and short descriptions. We used a simple HR schema with three tables: employees, salary, and studies.

### Finding 1: Simple questions return minimal and correct results
For a straightforward question about average salary, the model correctly returned only the salary table. It did not include unnecessary tables, which shows it understands the question scope and does not over-select.

### Finding 2: Multi-condition questions trigger correct multi-table selection
When the question involved both education and salary, the model correctly identified employees, salary, and studies as all three being necessary. It understood that filtering by degree requires the studies table even though the question was phrased naturally.

### Finding 3: Broad questions select all tables
When asked for a full employee profile including salary and education, the model returned all three tables. This is correct but worth noting — vague or broad questions will always pull in more tables than needed, which could be a problem in large databases with many tables.

### Conclusion
Using table names and short definitions is an efficient way to let the model pre-select relevant tables before generating a full SQL query. It works well for focused questions but broad questions may return too many tables. Adding more precise definitions helps the model make better selections.